## Background code

#### Imports

In [1]:
import os
import zlib
import time
from IPython.display import display, HTML
from Cryptodome.PublicKey import RSA as PyCryptoRSA
from Cryptodome.Cipher import PKCS1_OAEP
from classes.rsa import RSA 
from classes.png import PNG 

#### Helpers

In [2]:
def visualiseResults(img_path, encrypt_img_path, decrypt_img_path, keysize, title=None):
    if title is not None:
        title += ' '
    else:
        title = ''
    html = f"""
    <style>
        table {{
            margin: auto;
        }}
        th {{
            text-align: center;
            padding-bottom: 10px;
            font-size: 16px;
        }}
        td {{
            text-align: center;
            border: 1px inset grey;
        }}
        p {{
            text-align: center;
        }}
    </style>
    <p>{title}Keysize: {keysize}</p>
    <table>
    <tr>
        <th>Original</th>
        <th>Encrypted</th>
        <th>Decrypted</th>
    </tr>
    <tr>
        <td><img src="{img_path}" width="300"></td>
        <td><img src="{encrypt_img_path}" width="300"></td>
        <td><img src="{decrypt_img_path}" width="300"></td>
    </tr>
    </table>
    """
    display(HTML(html))

#### Keypair generation

In [3]:
keysize = 256
keypair = RSA.generateKeypair(keysize)
print(keypair)

((11359556787390358028163080864621472191858152949621378854340413834038335346227, 65537), (11359556787390358028163080864621472191858152949621378854340413834038335346227, 5855795451503667479797084455046337435648005610690099111085123222021652091585))


#### Path to image

In [4]:
filename = "dice.png"
img_path = f"media/{filename}"

## Porównanie szyfrowania danych bez oraz z dekompresją danych

### Szyfrowanie po dekompresji danych z kompresją przed zapisaniem (całość) dla dwóch rozmiarów kluczy - 16-bit & 512-bit 

In [5]:
keypair512 = RSA.generateKeypair(512)
keypair16 = RSA.generateKeypair(16)
print(f"512-bit Keypair - {keypair512}")
print(f" 16-bit Keypair - {keypair16}")

512-bit Keypair - ((633093742674171596018341062675005960357036579663247262891778701427915000301401375882811767677894460971298028857224073528695378120813327379369074309568581, 65537), (633093742674171596018341062675005960357036579663247262891778701427915000301401375882811767677894460971298028857224073528695378120813327379369074309568581, 597419006392441951632791929445550888998894826322441731026430291455330199408464610717020706856150791530572282892801441879413979685745403099747803478165873))
 16-bit Keypair - ((11929, 65537), (11929, 9773))


#### Encryption

In [6]:
##########################################################################
###########################512-bit encrypt################################
##########################################################################

png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)
pubkey = keypair512[0]

chunk_size = len(idat_chunks[0])
combined_data = PNG.concatDataChunks(idat_chunks)

try:
    decompressed_data = zlib.decompress(combined_data)
except zlib.error:
    decompressed_data = combined_data


encrypted_uncompressed = RSA.encryptData(decompressed_data, pubkey)
encrypted_compressed = zlib.compress(encrypted_uncompressed)
new_chunk = PNG.buildNewIDAT(encrypted_compressed)

encrypted_idat_chunks = [new_chunk]
all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, encrypted_idat_chunks)

timestamp = RSA.getTimeStamp()
encrypt_img_path512 = RSA.writeEncrypted(all_chunks, png_name, timestamp)
keyfile512 = RSA.saveKeys(keypair512, png_name, timestamp)

time.sleep(1)
##########################################################################
############################16-bit encrypt################################
##########################################################################

png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)
pubkey = keypair16[0]

chunk_size = len(idat_chunks[0])
combined_data = PNG.concatDataChunks(idat_chunks)

try:
    decompressed_data = zlib.decompress(combined_data)
except zlib.error:
    decompressed_data = combined_data


encrypted_uncompressed = RSA.encryptData(decompressed_data, pubkey)
encrypted_compressed = zlib.compress(encrypted_uncompressed)
new_chunk = PNG.buildNewIDAT(encrypted_compressed)

encrypted_idat_chunks = [new_chunk]
all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, encrypted_idat_chunks)

timestamp = RSA.getTimeStamp()
encrypt_img_path16 = RSA.writeEncrypted(all_chunks, png_name, timestamp)
keyfile16 = RSA.saveKeys(keypair16, png_name, timestamp)


#### Decryption


In [7]:
##########################################################################
############################16-bit decrypt################################
##########################################################################

keys = RSA.loadKeys(keyfile16)
privkey = keys[1]

png = PNG(encrypt_img_path16)
png_name = os.path.basename(encrypt_img_path16)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)

encrypted_idat_chunk = encrypted_idat_chunks[0]

encrypted_decompressed = zlib.decompress(encrypted_idat_chunk[8:-4])
decrypted_decompressed = RSA.decryptData(encrypted_decompressed, privkey)
decrypted_recompressed = zlib.compress(decrypted_decompressed)

decrypted_idat_chunks = []
for i in range(0, len(decrypted_recompressed), chunk_size):
    chunk_data = decrypted_recompressed[i:i+chunk_size]

    new_chunk = PNG.buildNewIDAT(chunk_data)
    decrypted_idat_chunks.append(new_chunk)

all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, decrypted_idat_chunks)

decrypt_img_path16 = RSA.writeDecrypted(all_chunks, png_name)

##########################################################################
###########################512-bit decrypt################################
##########################################################################

keys = RSA.loadKeys(keyfile512)
privkey = keys[1]

png = PNG(encrypt_img_path512)
png_name = os.path.basename(encrypt_img_path512)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)

encrypted_idat_chunk = encrypted_idat_chunks[0]

encrypted_decompressed = zlib.decompress(encrypted_idat_chunk[8:-4])
decrypted_decompressed = RSA.decryptData(encrypted_decompressed, privkey)
decrypted_recompressed = zlib.compress(decrypted_decompressed)

decrypted_idat_chunks = []
for i in range(0, len(decrypted_recompressed), chunk_size):
    chunk_data = decrypted_recompressed[i:i+chunk_size]

    new_chunk = PNG.buildNewIDAT(chunk_data)
    decrypted_idat_chunks.append(new_chunk)

all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, decrypted_idat_chunks)

decrypt_img_path512 = RSA.writeDecrypted(all_chunks, png_name)

#### Porównanie

In [8]:
visualiseResults(img_path, encrypt_img_path16, decrypt_img_path16, 16)
visualiseResults(img_path, encrypt_img_path512, decrypt_img_path512, 512)

Original,Encrypted,Decrypted
,,


Original,Encrypted,Decrypted
,,


### Szyfrowanie po dekompresji bez kompresji przed zapisaniem (całość)

#### Encrypt

In [9]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

chunk_size = len(idat_chunks[0])
combined_data = PNG.concatDataChunks(idat_chunks)

try:
    decompressed_data = zlib.decompress(combined_data)
except zlib.error:
    decompressed_data = combined_data

encrypted_idat_chunks = []

encrypted_uncompressed = RSA.encryptData(decompressed_data, pubkey)

new_chunk = PNG.buildNewIDAT(encrypted_uncompressed)
encrypted_idat_chunks.append(new_chunk)

all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, encrypted_idat_chunks)

timestamp = RSA.getTimeStamp()
encrypt_img_path = RSA.writeEncrypted(all_chunks, png_name, timestamp)
keyfile = RSA.saveKeys(keypair, png_name, timestamp)

#### Decrypt

In [10]:
keys = RSA.loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)

encrypted_idat_chunk = encrypted_idat_chunks[0]

decrypted_decompressed = RSA.decryptData(encrypted_idat_chunk[8:-4], privkey)
decrypted_recompressed = zlib.compress(decrypted_decompressed)

decrypted_idat_chunks = []
for i in range(0, len(decrypted_recompressed), chunk_size):
    chunk_data = decrypted_recompressed[i:i+chunk_size]

    new_chunk = PNG.buildNewIDAT(chunk_data)
    decrypted_idat_chunks.append(new_chunk)

all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, decrypted_idat_chunks)

decrypt_img_path = RSA.writeDecrypted(all_chunks, png_name)

#### Porównanie

In [11]:
visualiseResults(img_path, encrypt_img_path, decrypt_img_path, keysize)

Original,Encrypted,Decrypted
,,


### Szyfrowanie bez dekompresji danych

#### Jako całość

##### Encryption

In [12]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

chunk_size = len(idat_chunks[0])
combined_data = PNG.concatDataChunks(idat_chunks)

encrypted_data = RSA.encryptData(combined_data, pubkey)
new_chunk = PNG.buildNewIDAT(encrypted_data)

encrypted_idat_chunks = [new_chunk]
all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, encrypted_idat_chunks)

timestamp = RSA.getTimeStamp()
encrypt_img_path = RSA.writeEncrypted(all_chunks, png_name, timestamp)
keyfile = RSA.saveKeys(keypair, png_name, timestamp)

##### Decryption

In [13]:
keys = RSA.loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)

crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_idat = png.getDataChunks(crit_chunks)
encrypted_idat_chunk = encrypted_idat_chunks[0]

decrypted_data = RSA.decryptData(encrypted_idat_chunk[8:-4], privkey)

decrypted_idat_chunks = []
for i in range(0, len(decrypted_data), chunk_size):
    chunk_data = decrypted_data[i:i+chunk_size]

    new_chunk = PNG.buildNewIDAT(chunk_data)
    decrypted_idat_chunks.append(new_chunk)

all_chunks = png.buildFromChunks(
    anc_chunks, crit_chunks_no_idat, decrypted_idat_chunks)

decrypt_img_path = RSA.writeDecrypted(all_chunks, png_name)

#### Porównanie

In [14]:
visualiseResults(img_path, encrypt_img_path, decrypt_img_path, keysize)

Original,Encrypted,Decrypted
,,


#### Per chunki (standard ecb)

##### Encryption

In [15]:
encrypt_img_path, keyfile = RSA.encryptFileECB(img_path, keypair)

##### Decryption

In [16]:
decrypt_img_path = RSA.decryptFileECB(encrypt_img_path, keyfile)

#### Porównanie

In [17]:
visualiseResults(img_path, encrypt_img_path, decrypt_img_path, keysize)

Original,Encrypted,Decrypted
,,


## Porównanie metod szyfrowania: ECB (Electronic CodeBook) oraz CBC (Cypher Clock Chaining)

### ECB

#### Encryption

In [18]:
ecb_encrypted_img_path, keyfile_path =  RSA.encryptFileECB(img_path, keypair)

print(f"""Original png: {img_path}
Encrypted png: {ecb_encrypted_img_path}
Keyfile: {keyfile_path}
""")

Original png: media/dice.png
Encrypted png: files/20250614194302_dice_encrypted.png
Keyfile: files/20250614194302_dice_keys.txt



#### Decryption

In [19]:
ecb_decrypted_img_path =  RSA.decryptFileECB(
    ecb_encrypted_img_path, keyfile_path)

print(f"Decrypted png: {ecb_decrypted_img_path}")

Decrypted png: files/20250614194302_dice_decrypted.png


#### Porównanie

In [20]:
visualiseResults(img_path, ecb_encrypted_img_path, ecb_decrypted_img_path, keysize)

Original,Encrypted,Decrypted
,,


### CBC

#### Encryption

In [21]:
cbc_encrypted_img_path, keyfile_path, iv =  RSA.encryptFileCBC(img_path, keypair)

print(f"""Original png: {img_path}
Encrypted png: {cbc_encrypted_img_path}
Keyfile: {keyfile_path}
""")

Original png: media/dice.png
Encrypted png: files/20250614194302_dice_encrypted.png
Keyfile: files/20250614194302_dice_keys.txt



#### Decryption

In [22]:
cbc_decrypted_img_path =  RSA.decryptFileCBC(
    cbc_encrypted_img_path, keyfile_path, iv)

print(f"Decrypted png: {cbc_decrypted_img_path}")

Decrypted png: files/20250614194302_dice_decrypted.png


#### Porównanie

In [23]:
visualiseResults(img_path, cbc_encrypted_img_path, cbc_decrypted_img_path, keysize)

Original,Encrypted,Decrypted
,,


##### Data lengths check

In [24]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 179514 encrypted: 185324 decrypted: 179514


## Porównanie z gotową implementacją RSA

##### Keygen

In [25]:
keysize = 1024
custom_keypair = RSA.generateKeypair(keysize)
print(f""" Key:
n - {custom_keypair[0][0]}
e - {custom_keypair[0][1]}
d - {custom_keypair[1][1]}
""") 

 Key:
n - 74520928590597443280789398093044645824483599314807290019148060495525796029038674239405980851017207730592696183420017596018555539281520954798763582464379483747161680602169185091251326004468310811752385842858032882613687217531363315810583642433298251603171859374628240887119444679570088148449685398692543747053
e - 65537
d - 3995705373565457919781099911484487929371734562037212828284576416089806479485510494488191658307131360381200457581792603146454738011127525446127458211084264533974890684083019768122584056279156678155629968912398816146151432962286483423518858270409785910300086825242263514142582731922623628602937782776006852609



##### Encode/Decode

In [26]:
img_path = f"media/dice.png"

##########################################################################
###########################Our Implementation#############################
##########################################################################

# Encrypt
cbc_enc_start = time.time()
cbc_encrypted_img_path, keyfile_path, iv = RSA.encryptFileCBC(img_path, custom_keypair)
cbc_enc_end = time.time()

# Decrypt
cbc_dec_start = time.time()
cbc_decrypted_img_path = RSA.decryptFileCBC(cbc_encrypted_img_path, keyfile_path, iv)
cbc_dec_end = time.time()

##########################################################################
#######################Python package PyCryptodome########################
##########################################################################
png = PNG(img_path)
idat_chunks, _ = png.getDataChunks(png.getCriticalChunks())
idat_data = png.concatDataChunks(idat_chunks)

pycrypto_key = PyCryptoRSA.construct((
    custom_keypair[0][0], 
    custom_keypair[0][1], 
    custom_keypair[1][1]
    ))
cipher = PKCS1_OAEP.new(pycrypto_key)

byte_len = (custom_keypair[0][0].bit_length() + 7) // 8
max_chunk = cipher._key.size_in_bytes() - 42

# Encrypt
cbc_enc_std_start = time.time()
enc_data = bytearray()
for i in range(0, len(idat_data), max_chunk):
    chunk = idat_data[i:i+max_chunk]
    enc_data.extend(cipher.encrypt(chunk))
cbc_enc_std_end = time.time()

# Decrypt
cbc_dec_std_start = time.time()
dec_data = bytearray()
keysize_bytes = cipher._key.size_in_bytes()
for i in range(0, len(enc_data), keysize_bytes):
    chunk = enc_data[i:i+keysize_bytes]
    dec_data.extend(cipher.decrypt(chunk))
cbc_dec_std_end = time.time()

print(f"""Python implementation time decrypt: {cbc_enc_std_end - cbc_enc_std_start
:>4.3f} ms encrypt: {cbc_dec_std_end - cbc_dec_std_start:>4.3f} ms
Custom implementation time decrypt: {cbc_enc_end - cbc_enc_start
:>4.3f} ms encrypt: {cbc_dec_end - cbc_dec_start:>4.3f} ms""")

Python implementation time decrypt: 0.271 ms encrypt: 1.251 ms
Custom implementation time decrypt: 0.062 ms encrypt: 3.413 ms


##### Porównanie

In [27]:

std_encrypted_chunks = [PNG.buildNewIDAT(enc_data)]
std_decrypted_chunks = [PNG.buildNewIDAT(dec_data)]
anc_chunks = png.getAncillaryChunks()
crit_chunks = png.getCriticalChunks()
_, crit_no_idat = png.getDataChunks(crit_chunks)

timestamp = RSA.getTimeStamp()
std_enc_path = RSA.writeEncrypted(
    png.buildFromChunks(anc_chunks, crit_no_idat, std_encrypted_chunks),
    img_path, timestamp
)
time.sleep(1)
std_dec_path = RSA.writeDecrypted(
    png.buildFromChunks(anc_chunks, crit_no_idat, std_decrypted_chunks),
    std_enc_path
)

visualiseResults(
    img_path,
    cbc_encrypted_img_path,
    cbc_decrypted_img_path,
    keysize,
    "Nasza implementacja"
)

visualiseResults(
    img_path,
    std_enc_path,
    std_dec_path,
    keysize,
    "Implementacja PyCryptodome"
)

Original,Encrypted,Decrypted
,,


Original,Encrypted,Decrypted
,,


## Porównanie plików

In [28]:
files = ['dice', 'test', 'plte']
# files = ['dice']

for file in files:
    print(f"File: {file}.png")
    png = PNG(f"media/{file}.png")
    anc_chunks = png.getAncillaryChunks()
    crit_chunks = png.getCriticalChunks()
    print(f"Crit chunks: {len(crit_chunks)} Anc chunks: {len(anc_chunks)}")
    print("Crit chunk sizes: ")
    for chunk in crit_chunks:
        data_len = int.from_bytes(chunk[:4])
        chunk_type = chunk[4:8].decode('utf-8')
        print(f"{chunk_type}: {data_len}")


File: dice.png
Crit chunks: 3 Anc chunks: 0
Crit chunk sizes: 
IHDR: 13
IDAT: 179502
IEND: 0
File: test.png
Crit chunks: 10 Anc chunks: 8
Crit chunk sizes: 
IHDR: 13
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 7564
IEND: 0
File: plte.png
Crit chunks: 5 Anc chunks: 6
Crit chunk sizes: 
IHDR: 13
PLTE: 768
IDAT: 8192
IDAT: 2484
IEND: 0


In [29]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 179514 encrypted: 185324 decrypted: 179514
